In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 160)

In [2]:
# Defining functions

# ------------ Helper: safe numeric conversion
def to_num(s):
    return pd.to_numeric(s, errors="coerce")

# ------------ Helper: extract domain
def extract_domain(url):
    if pd.isna(url) or not isinstance(url, str) or not url.strip():
        return np.nan
    try:
        netloc = urlparse(url).netloc.lower()
        # strip "www."
        return re.sub(r"^www\.", "", netloc)
    except Exception:
        return np.nan

# ------------ Text cleaners (very light for EDA)
URL_RE = re.compile(r"https?://\S+|www\.\S+", flags=re.IGNORECASE)
MULTI_WS = re.compile(r"\s+")

def clean_text_basic(txt: str) -> str:
    if not isinstance(txt, str) or not txt:
        return ""
    txt = URL_RE.sub(" ", txt)
    txt = txt.replace("\u200b", " ")  # zero width
    txt = MULTI_WS.sub(" ", txt).strip()
    return txt

# ------------ Columns we expect (Meta API usually returns these)
REACTION_COLS = [
    "statistics.like_count", "statistics.love_count", "statistics.care_count",
    "statistics.haha_count", "statistics.wow_count", "statistics.sad_count",
    "statistics.angry_count"
]
COUNT_COLS = REACTION_COLS + ["statistics.comment_count", "statistics.share_count", "statistics.reaction_count", "statistics.views"]

In [3]:
def prepare_df(df: pd.DataFrame, brand: str, group: str) -> pd.DataFrame:
    """
    group ∈ {"operadora","geral"}
    brand ∈ {"oi","tim","vivo","claro"}
    """
    df = df.copy()

    # 1) Parse times
    for col in ["creation_time", "modified_time"]:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce", utc=True)

    # 2) Ensure numeric
    for c in COUNT_COLS:
        if c in df.columns:
            df[c] = to_num(df[c])

    # 3) Fallbacks
    if "statistics.reaction_count" not in df.columns:
        df["statistics.reaction_count"] = np.nan
    if "statistics.views" not in df.columns:
        df["statistics.views"] = np.nan
    if "statistics.comment_count" not in df.columns:
        df["statistics.comment_count"] = np.nan
    if "statistics.share_count" not in df.columns:
        df["statistics.share_count"] = np.nan

    # 4) Derived: total reactions (prefer the explicit sum, else the API's reaction_count)
    df["reactions_sum"] = df[REACTION_COLS].sum(axis=1, skipna=True) if set(REACTION_COLS).issubset(df.columns) else np.nan
    df["reactions"] = df["reactions_sum"].fillna(df["statistics.reaction_count"])

    # 5) Derived: engagements
    df["comments"] = df["statistics.comment_count"]
    df["shares"] = df["statistics.share_count"]
    df["views"] = df["statistics.views"]
    df["engagements"] = df[["reactions", "comments", "shares"]].sum(axis=1, skipna=True)

    # 6) Engagement rate (when views available and > 0)
    df["engagement_rate"] = np.where(df["views"] > 0, df["engagements"] / df["views"], np.nan)

    # 7) Text features
    if "text" not in df.columns:
        df["text"] = ""
    df["text_clean"] = df["text"].map(clean_text_basic)
    df["char_count"] = df["text_clean"].str.len()
    df["word_count"] = df["text_clean"].str.split().map(len)

    # 8) Link presence + domains
    link_cols = [c for c in ["link_attachment.link", "mcl_url", "link_attachment.name", "link_attachment.caption", "link_attachment.description"] if c in df.columns]
    df["has_link"] = False
    for c in link_cols:
        df["has_link"] = df["has_link"] | (~df[c].isna() & (df[c].astype(str).str.len() > 0))
    url_col = "link_attachment.link" if "link_attachment.link" in df.columns else ("mcl_url" if "mcl_url" in df.columns else None)
    if url_col:
        df["domain"] = df[url_col].map(extract_domain)
    else:
        df["domain"] = np.nan

    # 9) Content-type flags (photo/video/status/album/story)
    if "content_type" not in df.columns:
        df["content_type"] = np.nan
    for t in ["link", "photos", "videos", "albums", "status", "stories"]:
        df[f"is_{t}"] = df["content_type"].astype(str).str.contains(t, case=False, na=False)

    # 10) Time breakdown
    df["created_date"] = df["creation_time"].dt.tz_convert("UTC").dt.date if "creation_time" in df.columns else pd.NaT
    df["created_day"] = pd.to_datetime(df["created_date"], errors="coerce")
    df["created_hour"] = df["creation_time"].dt.tz_convert("UTC").dt.hour if "creation_time" in df.columns else np.nan
    df["created_weekday"] = df["creation_time"].dt.tz_convert("UTC").dt.day_name() if "creation_time" in df.columns else np.nan
    df["created_month"] = df["creation_time"].dt.tz_convert("UTC").dt.to_period("M").astype(str) if "creation_time" in df.columns else np.nan

    # 11) Owner info (if present)
    if "post_owner.username" not in df.columns:
        df["post_owner.username"] = np.nan
    if "surface.name" not in df.columns:
        df["surface.name"] = np.nan

    # 12) Dataset labels
    df["brand"] = brand
    df["group"] = group                     # "operadora" vs "geral"
    df["dataset"] = group + ":" + brand     # e.g., "geral:vivo"

    # 13) De-dup helpers
    if "id" not in df.columns:
        df["id"] = np.nan
    df["text_sig"] = df["text_clean"].str.lower().str[:280]  # rough signature to catch near-dup texts

    return df

In [4]:
# Salvando datasets em dicionários
operadoras = ['oi', 'vivo', 'claro', 'tim']

df_operadora = {op: pd.read_csv(f"data/operadora_{op}_ptbr.csv") for op in operadoras}
df_geral     = {op: pd.read_csv(f"data/{op}_geral.csv") for op in operadoras}

frames = []
for b, d in df_operadora.items():
    frames.append(prepare_df(d, brand=b, group="operadora"))
for b, d in df_geral.items():
    frames.append(prepare_df(d, brand=b, group="geral"))

all_df = pd.concat(frames, ignore_index=True)

/tmp/ipykernel_1055/4139963322.py:4: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_operadora = {op: pd.read_csv(f"data/operadora_{op}_ptbr.csv") for op in operadoras}
/tmp/ipykernel_1055/4139963322.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_geral     = {op: pd.read_csv(f"data/{op}_geral.csv") for op in operadoras}
/tmp/ipykernel_1055/4139963322.py:5: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_geral     = {op: pd.read_csv(f"data/{op}_geral.csv") for op in operadoras}
/tmp/ipykernel_1055/416244893.py:70: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df["created_month"] = df["creation_time"].dt.tz_convert("UTC").dt.to_period("M").astype(str) if "creation_time" in df.columns else np.nan
/tmp/ipykernel_1055/416244893.py:70: UserWarning: Converting to Perio

In [ ]:
def dataset_summary(df):
    # Basic counts
    base = df.groupby("dataset").agg(
        posts=("id", "count"),
        unique_texts=("text_sig", "nunique"),
        with_links=("has_link", lambda s: int(s.fillna(False).sum())),
        with_views=("views", lambda s: int((s > 0).fillna(False).sum()))
    )

    # Content type split
    ctype = (
        df.pivot_table(index="dataset", columns="content_type", values="id", aggfunc="count", fill_value=0)
        if "content_type" in df.columns else pd.DataFrame()
    )
    # Core stats
    stats = df.groupby("dataset").agg(
        reactions_mean=("reactions", "mean"),
        comments_mean=("comments", "mean"),
        shares_mean=("shares", "mean"),
        views_mean=("views", "mean"),
        engagements_mean=("engagements", "mean"),
        eng_rate_mean=("engagement_rate", "mean"),
        char_p50=("char_count", "median"),
        word_p50=("word_count", "median"),
        created_min=("created_day", "min"),
        created_max=("created_day", "max")
    )

    # Top domains (per dataset)
    top_domains = (
        df.dropna(subset=["domain"])
          .groupby(["dataset","domain"]).size()
          .reset_index(name="n")
          .sort_values(["dataset","n"], ascending=[True, False])
    )

    # Top authors (per dataset)
    top_authors = (
        df.dropna(subset=["post_owner.username"])
          .groupby(["dataset", "post_owner.username"]).size()
          .reset_index(name="posts")
          .sort_values(["dataset","posts"], ascending=[True, False])
    )

    summary = base.join(stats, how="left").sort_values("dataset", )
    return summary, ctype, top_domains, top_authors

summary, ctype_split, top_domains, top_authors = dataset_summary(all_df)

In [10]:
# Overall numeric describe (after coercion)
numeric_cols = [c for c in all_df.columns if pd.api.types.is_numeric_dtype(all_df[c])]
overall_describe = all_df[numeric_cols].describe(percentiles=[.5, .9, .95]).T

print("=== Datasets summary ===")
display(summary)
print("\n=== Content-type cross-tab ===")
display(ctype_split.fillna(0).astype(int))
print("\n=== Numeric columns describe (overall) ===")
display(overall_describe)

=== Datasets summary ===


,posts,unique_texts,with_links,with_views,reactions_mean,comments_mean,shares_mean,views_mean,engagements_mean,eng_rate_mean,char_p50,word_p50,created_min,created_max
dataset,,,,,,,,,,,,,,
geral:claro,85779,57664,85779,73841,260.002402,36.773639,31.969748,17272.294796,328.745789,0.024167,865.0,143.0,2025-07-04,2025-08-19
geral:oi,87139,28553,87139,80226,513.899943,83.021184,19.363496,17668.425074,616.284622,0.046172,43.0,10.0,2025-04-20,2025-08-19
geral:tim,86267,56379,86267,71692,182.458565,14.817045,27.934158,18839.190356,225.209767,0.017506,321.0,55.0,2022-09-19,2025-08-18
geral:vivo,85264,55842,85264,70144,201.858054,55.488296,22.516887,12690.542456,279.863238,0.049912,359.0,59.0,2025-07-26,2025-08-19
operadora:claro,13942,10289,13942,8269,82.645962,30.019222,18.699541,11729.234611,131.364725,0.014963,678.0,111.0,2010-07-08,2025-08-06
operadora:oi,14343,10356,14343,8284,57.336610,24.255107,15.219759,11482.882062,96.811476,0.013909,422.0,69.0,2010-07-28,2025-08-06
operadora:tim,13252,8860,13252,6468,60.567763,27.574328,17.557350,9191.465986,105.699442,0.013538,323.0,54.0,2010-12-21,2025-08-06
operadora:vivo,18636,12560,18636,11077,111.825499,28.181691,20.090416,16451.583371,160.097607,0.015043,409.5,68.0,2010-07-01,2025-08-06



=== Content-type cross-tab ===


content_type,albums,links,photos,reshare,status,videos
dataset,,,,,,
geral:claro,10621,2753,48739,996,2193,20250
geral:oi,4544,842,57784,903,3356,19255
geral:tim,20711,10087,31025,632,3019,20415
geral:vivo,8145,6995,31917,1678,1404,34689
operadora:claro,1047,1687,6401,52,3818,845
operadora:oi,882,2397,6392,40,3946,599
operadora:tim,2558,1843,4530,34,3707,516
operadora:vivo,2906,1998,7638,391,4139,1431



=== Numeric columns describe (overall) ===


,count,mean,std,min,50%,90%,95%,max
id,404622.0,2.214800e+15,4.271727e+15,1.522044e+14,1.288036e+15,2.730857e+15,4.158760e+15,3.469424e+16
shared_post_id,4940.0,1.939184e+15,3.549042e+15,3.983335e+14,1.277851e+15,2.397891e+15,4.053536e+15,3.104457e+16
statistics.angry_count,402158.0,8.549103e-01,1.854913e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,3.915000e+03
statistics.care_count,402158.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
statistics.comment_count,404631.0,4.462268e+01,6.509617e+02,0.000000e+00,1.000000e+00,5.600000e+01,1.360000e+02,1.764660e+05
statistics.haha_count,402158.0,6.568916e+00,1.264685e+02,0.000000e+00,0.000000e+00,1.000000e+00,6.000000e+00,2.080800e+04
statistics.like_count,402158.0,2.221840e+02,1.915489e+03,0.000000e+00,1.200000e+01,2.540000e+02,6.690000e+02,3.877630e+05
statistics.love_count,402158.0,2.732253e+01,3.001907e+02,0.000000e+00,0.000000e+00,2.800000e+01,8.100000e+01,6.955800e+04
statistics.reaction_count,402225.0,2.637507e+02,2.193195e+03,0.000000e+00,1.300000e+01,3.070000e+02,8.080000e+02,3.989060e+05
statistics.sad_count,402158.0,2.459812e+00,6.549783e+01,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,1.559300e+04


In [5]:
for op in operadoras:
    print(f"{op.upper()}:")
    print(f"  Operadora: {df_operadora[op].shape[0]} linhas")
    print(f"  Geral:     {df_geral[op].shape[0]} linhas\n")

for op in operadoras:
    # Description of dataframes
    display(df_operadora[op].describe(include='all'))
    display(df_geral[op].describe(include='all'))

OI:
  Operadora: 11057 linhas
  Geral:     30001 linhas

TIM:
  Operadora: 9271 linhas
  Geral:     59112 linhas

VIVO:
  Operadora: 13361 linhas
  Geral:     58183 linhas

CLARO:
  Operadora: 11007 linhas
  Geral:     60628 linhas



,activities,content_type,creation_time,id,is_branded_content,lang,link_attachment.caption,link_attachment.description,link_attachment.link,link_attachment.name,...,statistics.sad_count,statistics.share_count,statistics.views,statistics.views_date_last_refreshed,statistics.wow_count,surface.type,surface.id,surface.name,surface.username,text
count,0.0,10990,11057,1.105700e+04,11057,11057,2217,2011,2265,2280,...,11027.000000,11057.000000,6.364000e+03,6364,11027.000000,11057,1.105700e+04,11055,10525,11056
unique,NaN,6,11055,NaN,2,1,776,1852,2180,2040,...,NaN,NaN,NaN,8,NaN,3,NaN,4157,3936,11056
top,NaN,photos,2017-02-28T16:06:42+00:00,NaN,False,pt,minhaoperadora.com.br,Italiano Salgados - Goiânia - Aceitamos encome...,tel:+556232564293,Italiano Salgados,...,NaN,NaN,NaN,2025-07-28,NaN,page,NaN,Italiano Salgados,italianosalgadosgoiania,Super Combo Bolo Caseiro Vulcao 🍮\n\n60 Unid. ...
freq,NaN,4815,2,NaN,11055,11057,266,25,18,25,...,NaN,NaN,NaN,6210,NaN,10391,NaN,1072,1072,1
mean,NaN,NaN,NaN,2.022315e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,0.638614,16.536312,9.146053e+03,NaN,0.890360,NaN,1.376716e+15,NaN,NaN,NaN
std,NaN,NaN,NaN,3.720891e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,8.552496,164.133692,7.573259e+04,NaN,14.321876,NaN,1.926622e+15,NaN,NaN,NaN
min,NaN,NaN,NaN,3.289259e+14,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,1.010000e+02,NaN,0.000000,NaN,3.230225e+13,NaN,NaN,NaN
25%,NaN,NaN,NaN,7.812097e+14,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,5.960000e+02,NaN,0.000000,NaN,7.311277e+14,NaN,NaN,NaN
50%,NaN,NaN,NaN,1.265027e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,1.000000,1.401500e+03,NaN,0.000000,NaN,9.884471e+14,NaN,NaN,NaN
75%,NaN,NaN,NaN,1.715103e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,4.000000,3.803750e+03,NaN,0.000000,NaN,1.432272e+15,NaN,NaN,NaN


,activities,content_type,creation_time,id,is_branded_content,lang,link_attachment.caption,link_attachment.description,link_attachment.link,link_attachment.name,...,statistics.sad_count,statistics.share_count,statistics.views,statistics.views_date_last_refreshed,statistics.wow_count,surface.type,surface.id,surface.name,surface.username,text
count,6,29817,30001,3.000100e+04,30001,30001,649,591,838,854,...,29625.000000,30001.000000,2.802200e+04,28022,29625.000000,30001,3.000100e+04,30001,18915,30001
unique,6,6,29901,NaN,2,1,265,417,801,658,...,NaN,NaN,NaN,9,NaN,4,NaN,10492,7196,30001
top,"[{""name"":""Casino Classic Slots Community"",""typ...",photos,2025-07-22T02:14:17+00:00,NaN,False,pt,youtube.com,"Quando isso acontece, geralmente é porque o do...",https://www.facebook.com/reel/611625874710549/,Este conteúdo não está disponível no momento,...,NaN,NaN,NaN,2025-08-11,NaN,page,NaN,baixinha do interior,gatopoles,"Oi coração boa tarde, tudo bem com você 😚."
freq,1,15203,3,NaN,29999,30001,110,67,5,67,...,NaN,NaN,NaN,19081,NaN,23004,NaN,698,424,1
mean,NaN,NaN,NaN,2.211817e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,3.964152,28.714510,2.168670e+04,NaN,2.131679,NaN,1.597870e+15,NaN,NaN,NaN
std,NaN,NaN,NaN,4.230600e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,62.584341,209.285479,1.764470e+05,NaN,11.159834,NaN,2.696524e+15,NaN,NaN,NaN
min,NaN,NaN,NaN,4.286261e+14,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,1.010000e+02,NaN,0.000000,NaN,1.580223e+13,NaN,NaN,NaN
25%,NaN,NaN,NaN,7.977960e+14,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,7.122500e+02,NaN,0.000000,NaN,7.018994e+14,NaN,NaN,NaN
50%,NaN,NaN,NaN,1.289866e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,1.000000,2.401000e+03,NaN,0.000000,NaN,1.093966e+15,NaN,NaN,NaN
75%,NaN,NaN,NaN,1.797198e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,9.000000,9.826750e+03,NaN,1.000000,NaN,1.643737e+15,NaN,NaN,NaN


,activities,content_type,creation_time,id,is_branded_content,lang,link_attachment.caption,link_attachment.description,link_attachment.link,link_attachment.name,...,statistics.sad_count,statistics.share_count,statistics.views,statistics.views_date_last_refreshed,statistics.wow_count,surface.type,surface.id,surface.name,surface.username,text
count,0.0,9213,9271,9.271000e+03,9271,9271,1753,1582,1757,1771,...,9248.000000,9271.000000,5.115000e+03,5115,9248.000000,9271,9.271000e+03,9268,8816,9271
unique,NaN,6,9264,NaN,2,1,750,1496,1711,1614,...,NaN,NaN,NaN,7,NaN,4,NaN,4144,3929,9271
top,NaN,photos,2012-09-01T19:22:45+00:00,NaN,False,pt,minhaoperadora.com.br,"Quando isso acontece, geralmente é porque o do...",https://www.instagram.com/jrmunews,Este conteúdo não está disponível no momento,...,NaN,NaN,NaN,2025-07-28,NaN,page,NaN,#Minha Operadora,minhaoperadora,DIA DOS PAIS: GASTO MÉDIO DOS MINEIROS COM PRE...
freq,NaN,3722,3,NaN,9266,9271,252,9,8,9,...,NaN,NaN,NaN,5005,NaN,8491,NaN,269,269,1
mean,NaN,NaN,NaN,2.089943e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,0.438149,21.128573,9.349261e+03,NaN,0.634191,NaN,1.396839e+15,NaN,NaN,NaN
std,NaN,NaN,NaN,3.929764e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,7.822284,255.432789,7.092473e+04,NaN,6.622166,NaN,2.194942e+15,NaN,NaN,NaN
min,NaN,NaN,NaN,3.015797e+14,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,1.010000e+02,NaN,0.000000,NaN,2.000223e+13,NaN,NaN,NaN
25%,NaN,NaN,NaN,7.795990e+14,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,7.050000e+02,NaN,0.000000,NaN,5.546723e+14,NaN,NaN,NaN
50%,NaN,NaN,NaN,1.263443e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,1.000000,1.805000e+03,NaN,0.000000,NaN,9.884471e+14,NaN,NaN,NaN
75%,NaN,NaN,NaN,1.726370e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,4.000000,5.639500e+03,NaN,0.000000,NaN,1.417925e+15,NaN,NaN,NaN


,activities,content_type,creation_time,id,is_branded_content,lang,link_attachment.caption,link_attachment.description,link_attachment.link,link_attachment.name,...,statistics.sad_count,statistics.share_count,statistics.views,statistics.views_date_last_refreshed,statistics.wow_count,surface.type,surface.id,surface.name,surface.username,text
count,0.0,58835,59112,5.911200e+04,59112,59112,8870,8143,8965,8992,...,58687.000000,59112.000000,5.537300e+04,55373,58687.000000,59112,5.911200e+04,59112,53832,59112
unique,NaN,6,58973,NaN,2,1,1899,7258,8495,8058,...,NaN,NaN,NaN,9,NaN,4,NaN,14172,12507,59112
top,NaN,photos,2025-06-02T14:46:27+00:00,NaN,False,pt,youtube.com,"Quando isso acontece, geralmente é porque o do...",https://www.instagram.com/jrmunews,JRMUNEWS Retrô (@jrmunews) • Instagram photos ...,...,NaN,NaN,NaN,2025-08-11,NaN,page,NaN,Educadora Online,EducadoraOnline,Um homem foi baleado nesse domingo (17) na ave...
freq,NaN,24467,7,NaN,59088,59112,374,70,89,89,...,NaN,NaN,NaN,52555,NaN,54070,NaN,1752,1752,1
mean,NaN,NaN,NaN,2.194606e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,2.053334,28.394793,1.792901e+04,NaN,0.934653,NaN,2.054516e+15,NaN,NaN,NaN
std,NaN,NaN,NaN,4.250594e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,43.358293,476.790628,1.506409e+05,NaN,12.657480,NaN,4.010893e+15,NaN,NaN,NaN
min,NaN,NaN,NaN,1.522044e+14,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,1.010000e+02,NaN,0.000000,NaN,1.910224e+13,NaN,NaN,NaN
25%,NaN,NaN,NaN,7.959115e+14,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,5.770000e+02,NaN,0.000000,NaN,6.370783e+14,NaN,NaN,NaN
50%,NaN,NaN,NaN,1.286198e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,1.000000,1.730000e+03,NaN,0.000000,NaN,1.046252e+15,NaN,NaN,NaN
75%,NaN,NaN,NaN,1.767933e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,4.000000,6.588000e+03,NaN,0.000000,NaN,1.676998e+15,NaN,NaN,NaN


,activities,content_type,creation_time,id,is_branded_content,lang,link_attachment.caption,link_attachment.description,link_attachment.link,link_attachment.name,...,statistics.sad_count,statistics.share_count,statistics.views,statistics.views_date_last_refreshed,statistics.wow_count,surface.type,surface.id,surface.name,surface.username,text
count,0.0,13258,13361,1.336100e+04,13361,13361,1899,1676,1899,1933,...,13308.000000,13361.000000,8.500000e+03,8500,13308.000000,13361,1.336100e+04,13359,12698,13360
unique,NaN,6,13355,NaN,2,1,882,1553,1837,1755,...,NaN,NaN,NaN,8,NaN,4,NaN,5614,5278,13360
top,NaN,photos,2019-04-12T01:29:20+00:00,NaN,False,pt,minhaoperadora.com.br,"Quando isso acontece, geralmente é porque o do...",https://www.instagram.com/jrmunews,Este conteúdo não está disponível no momento,...,NaN,NaN,NaN,2025-07-28,NaN,page,NaN,Italiano Salgados,italianosalgadosgoiania,Na tarde desta quarta feira milhares de client...
freq,NaN,5999,2,NaN,13357,13361,238,19,10,19,...,NaN,NaN,NaN,8194,NaN,12320,NaN,405,405,1
mean,NaN,NaN,NaN,2.057756e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,0.933574,19.675623,1.563702e+04,NaN,0.792155,NaN,1.394910e+15,NaN,NaN,NaN
std,NaN,NaN,NaN,3.791220e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,21.508129,162.726652,8.557925e+04,NaN,5.902295,NaN,2.109365e+15,NaN,NaN,NaN
min,NaN,NaN,NaN,1.882343e+14,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,1.010000e+02,NaN,0.000000,NaN,3.490226e+13,NaN,NaN,NaN
25%,NaN,NaN,NaN,7.786032e+14,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,7.815000e+02,NaN,0.000000,NaN,5.732577e+14,NaN,NaN,NaN
50%,NaN,NaN,NaN,1.257378e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,1.000000,2.148000e+03,NaN,0.000000,NaN,9.884471e+14,NaN,NaN,NaN
75%,NaN,NaN,NaN,1.730294e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,6.000000,7.021500e+03,NaN,0.000000,NaN,1.425738e+15,NaN,NaN,NaN


,activities,content_type,creation_time,id,is_branded_content,lang,link_attachment.caption,link_attachment.description,link_attachment.link,link_attachment.name,...,statistics.sad_count,statistics.share_count,statistics.views,statistics.views_date_last_refreshed,statistics.wow_count,surface.type,surface.id,surface.name,surface.username,text
count,90,57873,58183,5.817700e+04,58183,58183,5187,4837,5293,5318,...,57788.000000,58183.000000,4.948300e+04,49483,57788.000000,58183,5.818300e+04,58183,52443,58182
unique,16,6,56250,NaN,2,1,1106,4127,5115,4866,...,NaN,NaN,NaN,9,NaN,4,NaN,18413,16010,58182
top,"[{""name"":""777 Slots Casino"",""type"":""streaming""}]",videos,2025-08-15T11:27:44+00:00,NaN,False,pt,youtube.com,"Quando isso acontece, geralmente é porque o do...",http://timao.me/impeachmentaovivo,Este conteúdo não está disponível no momento,...,NaN,NaN,NaN,2025-08-11,NaN,page,NaN,Basílica Sagrada Família,basilicadasagradafamilia,Lady Gaga no Dress to Impress? 👗✨\n\nBora conf...
freq,33,23906,10,NaN,58177,58183,1403,49,13,49,...,NaN,NaN,NaN,17631,NaN,52913,NaN,222,222,1
mean,NaN,NaN,NaN,2.274055e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,3.474510,22.301944,1.217358e+04,NaN,1.051274,NaN,1.503956e+15,NaN,NaN,NaN
std,NaN,NaN,NaN,4.437159e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,55.771052,232.671677,1.321814e+05,NaN,19.564123,NaN,2.138608e+15,NaN,NaN,NaN
min,NaN,NaN,NaN,4.700715e+14,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,1.010000e+02,NaN,0.000000,NaN,1.500225e+13,NaN,NaN,NaN
25%,NaN,NaN,NaN,7.971880e+14,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,4.480000e+02,NaN,0.000000,NaN,7.129611e+14,NaN,NaN,NaN
50%,NaN,NaN,NaN,1.293636e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,1.000000,1.319000e+03,NaN,0.000000,NaN,1.062707e+15,NaN,NaN,NaN
75%,NaN,NaN,NaN,1.789012e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,5.000000,4.827500e+03,NaN,0.000000,NaN,1.513428e+15,NaN,NaN,NaN


,activities,content_type,creation_time,id,is_branded_content,lang,link_attachment.caption,link_attachment.description,link_attachment.link,link_attachment.name,...,statistics.sad_count,statistics.share_count,statistics.views,statistics.views_date_last_refreshed,statistics.wow_count,surface.type,surface.id,surface.name,surface.username,text
count,0.0,10945,11007,1.100700e+04,11007,11007,1570,1414,1593,1613,...,10929.000000,11007.000000,6.859000e+03,6859,10929.000000,11007,1.100700e+04,11005,10406,11006
unique,NaN,6,11001,NaN,2,1,720,1292,1507,1437,...,NaN,NaN,NaN,8,NaN,4,NaN,4719,4446,11006
top,NaN,photos,2012-08-24T20:24:58+00:00,NaN,False,pt,minhaoperadora.com.br,Italiano Salgados - Goiânia - Aceitamos encome...,tel:+556232564293,Italiano Salgados,...,NaN,NaN,NaN,2025-07-28,NaN,page,NaN,Italiano Salgados,italianosalgadosgoiania,uma entrevista hoje do deputado Eduardo\nBolso...
freq,NaN,5358,3,NaN,10983,11007,196,25,18,25,...,NaN,NaN,NaN,6634,NaN,10147,NaN,1102,1102,1
mean,NaN,NaN,NaN,2.023472e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,0.982981,19.937676,1.215278e+04,NaN,1.063318,NaN,1.364817e+15,NaN,NaN,NaN
std,NaN,NaN,NaN,3.716049e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,22.157027,171.905207,9.427288e+04,NaN,11.251347,NaN,1.872735e+15,NaN,NaN,NaN
min,NaN,NaN,NaN,2.723178e+14,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,1.010000e+02,NaN,0.000000,NaN,1.862553e+14,NaN,NaN,NaN
25%,NaN,NaN,NaN,7.837349e+14,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,6.715000e+02,NaN,0.000000,NaN,7.241228e+14,NaN,NaN,NaN
50%,NaN,NaN,NaN,1.253970e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,1.000000,1.650000e+03,NaN,0.000000,NaN,9.884471e+14,NaN,NaN,NaN
75%,NaN,NaN,NaN,1.722567e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,4.000000,4.762500e+03,NaN,0.000000,NaN,1.389382e+15,NaN,NaN,NaN


,activities,content_type,creation_time,id,is_branded_content,lang,link_attachment.caption,link_attachment.description,link_attachment.link,link_attachment.name,...,statistics.sad_count,statistics.share_count,statistics.views,statistics.views_date_last_refreshed,statistics.wow_count,surface.type,surface.id,surface.name,surface.username,text
count,0.0,60435,60628,6.062800e+04,60628,60628,2331,2081,2470,2477,...,60146.000000,60628.000000,5.412200e+04,54122,60146.000000,60628,6.062800e+04,60628,51417,60628
unique,NaN,6,59327,NaN,2,1,899,1929,2418,2345,...,NaN,NaN,NaN,9,NaN,4,NaN,23985,20779,60628
top,NaN,photos,2025-08-05T02:20:11+00:00,NaN,False,pt,jornalcidade.net,"Quando isso acontece, geralmente é porque o do...",https://bit.ly/4c4zK0p,Este conteúdo não está disponível no momento,...,NaN,NaN,NaN,2025-08-11,NaN,page,NaN,Templo N2,TemploN2,OPINIÃO:\nDescumprir Lei Magnitsky deve custar...
freq,NaN,29614,5,NaN,60616,60628,142,36,6,36,...,NaN,NaN,NaN,29368,NaN,53255,NaN,493,493,1
mean,NaN,NaN,NaN,2.273878e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,3.110182,25.927030,1.549218e+04,NaN,1.095418,NaN,1.536102e+15,NaN,NaN,NaN
std,NaN,NaN,NaN,4.433414e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,52.916238,298.448009,8.232411e+04,NaN,32.442779,NaN,2.397323e+15,NaN,NaN,NaN
min,NaN,NaN,NaN,4.696906e+14,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,1.010000e+02,NaN,0.000000,NaN,1.720223e+13,NaN,NaN,NaN
25%,NaN,NaN,NaN,7.960241e+14,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,5.270000e+02,NaN,0.000000,NaN,6.914036e+14,NaN,NaN,NaN
50%,NaN,NaN,NaN,1.292603e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,1.000000,1.748000e+03,NaN,0.000000,NaN,1.051756e+15,NaN,NaN,NaN
75%,NaN,NaN,NaN,1.788792e+15,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,6.000000,7.152500e+03,NaN,0.000000,NaN,1.538031e+15,NaN,NaN,NaN


In [21]:
# df_operadora['oi'].describe(include='all').to_excel("oi_operadora_describe.xlsx", index=True)

df_operadora['oi'][['id', 'match_type']].head(20)

df_operadora['oi'][['id', 'match_type']].groupby('match_type').count()

,id
match_type,
"[""post_text""]",14343
